In [1]:
from wallcheb.qtmlib.circuits.lcu import LCUMultiplexorBox
from guppylang import guppy
from guppylang.std.builtins import comptime
from guppylang.std.quantum import qubit, measure_array
from guppylang.std.builtins import array, exit
from typing import Callable


In [2]:
from wallcheb.operators import generate_pytket_hvs_hubbard

u = 1
n_sites = 2
m = 5
product_block_encoding_qpo = generate_pytket_hvs_hubbard(u, n_sites, m)
n_state_qubits = 2*n_sites


/Users/nathanfitzpatric/Github/wall_chebyshev/wallcheb/operators/hubbard_model.py:54: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[i,j] = hmat[subset[i], subset[j]]


In [3]:
n_prep_qubits = LCUMultiplexorBox(product_block_encoding_qpo[0], n_state_qubits).n_prepare_qubits

In [4]:
from pytket.passes import AutoRebase
from pytket import OpType
from pytket.passes import DecomposeBoxes


def build_multiplexor_lcu(ham, n_state_qubits, ind):
    multiplexor_lcu = LCUMultiplexorBox(ham, n_state_qubits)
    

    circ = multiplexor_lcu.get_circuit()
    DecomposeBoxes().apply(circ)
    rebase = AutoRebase({OpType.CX, OpType.Rz, OpType.H, OpType.CCX})
    rebase.apply(circ)

    from guppylang import guppy

    qlibs_multiplexor_lcu = guppy.load_pytket(f"qlibs_multiplexor_lcu_{ind}", circ)
    return qlibs_multiplexor_lcu

In [5]:
def guppy_multiplexor_lcu() -> array[Callable[[array[qubit, comptime(n_prep_qubits)], array[qubit, comptime(n_state_qubits)]],None], comptime(m)]:
    u = 1
    n_sites = 2
    m = 5

    product_block_encoding_qpo = generate_pytket_hvs_hubbard(int(u), int(n_sites), int(m))
    n_state_qubits = 2*n_sites
    guppy_circuits = [build_multiplexor_lcu(qpo, n_state_qubits, i) for i, qpo in enumerate(product_block_encoding_qpo)]
    return guppy_circuits

In [6]:
guppy_circuits = guppy_multiplexor_lcu()

In [8]:
from guppylang.std.builtins import comptime
from guppylang.std.quantum import qubit, discard_array, measure, measure_array
from hugr.qsystem.result import QsysResult
from guppylang.std.builtins import result, array, exit
from typing import Callable

@guppy
def product_block_encoding(prod_block_encoding: array[Callable[[array[qubit, comptime(n_prep_qubits)], array[qubit, comptime(n_state_qubits)]], None], comptime(m)], state_qreg: array[qubit, comptime(n_state_qubits)]) -> None:
    

    for prod_block in prod_block_encoding.copy():

        prep_qreg = array(qubit() for _ in range(comptime(n_prep_qubits)))
        prod_block(prep_qreg, state_qreg)

        outcome = measure_array(prep_qreg)
        # result("c", measure_array(prep_qubits))
        for b in outcome: 
            if b:
                exit("circuit failed",1)


In [9]:
@guppy
def main() -> None:
    """Main function to run the multiplexor LCU circuit."""
    state_qreg = array(qubit() for _ in range(comptime(n_state_qubits)))
    # The pytket function only acts on arrays

 
    prod = comptime(guppy_circuits)

    product_block_encoding(prod, state_qreg)

    result('c',measure_array(state_qreg))

compiled_hugr = guppy.compile(main)


Error: Unsupported Python expression (at <In [9]>:8:20)
  | 
6 | 
7 | 
8 |     prod = comptime(guppy_circuits)
  |                     ^^^^^^^^^^^^^^ Expression of type `<class 'list'>` is not supported

Guppy compilation failed due to 1 previous error


In [8]:
from hugr.qsystem.result import QsysResult
from selene_sim import build, Quest

n_shots = 80000
runner = build(compiled_hugr)
shots = QsysResult(
    runner.run_shots(
        Quest(random_seed=2), n_qubits=multiplexor_lcu.n_qubits, n_shots=n_shots
    )
)

print(f'Final state:{shots.register_counts()["c"]}, when initial state is |000>')

shots_counts = shots.register_counts()["c"]
success_prob = sum(shots_counts.values())/ n_shots
print(f"Success probability (from circuits): {success_prob:.4f}")

Final state:Counter({'0000': 537, '1000': 37, '0001': 32, '0100': 8, '0010': 8, '1001': 4, '0011': 4, '0110': 3, '1010': 2, '1100': 2, '1011': 1, '0111': 1, '0101': 1}), when initial state is |000>
Success probability (from circuits): 0.0080
Success probability (from circuits): 0.0080
